In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

import jupyter_dash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datetime import datetime as dt, date, time, timedelta

from plotly_calplot import calplot

from scipy.stats import pearsonr

# https://github.com/d3/d3-3.x-api-reference/blob/master/Time-Formatting.md#format

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [3]:
# pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv('data/df2_shifted.csv')
df

,Date,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,concert_event_counts,concert_event_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_dur

# Scatterplots

In [20]:
def make_scatter_dashboard(df=df, series1=None, series2=None):
    
    x = df[series1]
    y = df[series2]
    # rolling = 7
    # y2 = round(y.rolling(rolling).mean(), 2)   
    title= f"{x.name} vs. {y.name}"
    
    app = jupyter_dash.JupyterDash(__name__, external_stylesheets=external_stylesheets, title=f"{title} Dashboard")
    
    fig = px.scatter(df, x=x, y=y, trendline="ols", marginal_x="violin", marginal_y="violin")

    fig.update_layout(title=dict(text=title, 
                                 yanchor="top",
                                 y=.98,
                                 xanchor="left",
                                 x=.075),
                      legend=dict(orientation="h",
                                  yanchor="bottom",
                                  y=1.02,
                                  xanchor="right",
                                  x=1),
                      width=1000)

    fig.update_xaxes(title_text=f"{x.name}")
    fig.update_yaxes(title_text=f"{y.name}")

    
    results = px.get_trendline_results(fig)
    print(results.px_fit_results.iloc[0].summary())

    app.layout = html.Div([
        dcc.Graph(
            figure=fig,
        )
    ])

    title = title.replace('/', '_')
    # fig.write_image(f"data/images/combined/scatterplots/{title}.png")
    
    if __name__ == '__main__':
        app.run_server(mode='external', port=8020, debug=True)

In [21]:
make_scatter_dashboard(df=df, series1='Mood', series2='gym_durations_seconds')

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     31.40
Date:                Mon, 10 Apr 2023   Prob (F-statistic):           2.62e-08
Time:                        01:04:00   Log-Likelihood:                -10404.
No. Observations:                1165   AIC:                         2.081e+04
Df Residuals:                    1163   BIC:                         2.082e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -142.3954    261.094     -0.545      0.5

# Correlations Heatmaps

In [48]:
df_columns = [i for i in df.columns]

In [72]:
vitals = ['Step Count',
          'Resting Heart Rate',
          'Number of Floors',
          'Heart Rate Count',
          'Heart Rate Mean',
          'Heart Rate STD',
          'Heart Rate Minimum',
          'Heart Rate 25%',
          'Heart Rate Median',
          'Heart Rate 75%',
          'Heart Rate Maximum',]
df_counts = [i for i in df.columns if i.endswith('counts')]
df_durations_seconds = [i for i in df.columns if i.endswith('seconds')]
df_durations = [i for i in df.columns if i.endswith('durations')]
df_durations_seconds_7_day_rolling_mean = [i for i in df.columns if i.endswith('mean')]
df_shifted = [i for i in df.columns if i.endswith('shifted')]

In [73]:
df_columns_to_remove = ['Date', 'Timestamp', 'Pain Location', 'Notes'] + vitals
df_columns_to_remove = df_columns_to_remove + df_counts + df_durations_seconds + df_durations + df_durations_seconds_7_day_rolling_mean +df_shifted

In [74]:
df_columns = [item for item in df_columns if item not in df_columns_to_remove]
df_columns

['Mood',
 'Pain',
 'Tired',
 'Sick',
 'Anxiousness',
 'Depressed',
 'Suicidal',
 'Self-Harm',
 'Thoughts of Death',
 'Repetitive Thoughts',
 'Thoughts of Worthlessness',
 'Thoughts of Hopelessness',
 'Brain Sludge',
 'Sluggish',
 'Restless',
 'Existential Crisis',
 'Loss of Interest',
 'Self Isolation',
 'Paranoia',
 'Elevated',
 'Irritable/Angry',
 'Loneliness',
 'Libido',
 'Weight',
 'Weird',
 'Month',
 'Day',
 'Weekday',
 'Week']

In [75]:
def correlations(df=df):
    df.corr(method='pearson')
    
    globals()['rho'] = df.corr().round(3)
    globals()['pval'] = df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
    globals()['p'] = globals()['pval'].applymap(lambda x: ''.join(['*' for t in [0.001, 0.01, 0.05] if x<=t])) # can add more * for even smaller p value's e.g. 0.00001, 0.0001, in the list 
    globals()['correlation_matrix'] = globals()['rho'].astype(str) + p
    
    # export 
    globals()['rho'].reset_index().to_csv('data/rho.csv', index=None, encoding='utf-8')
    globals()['pval'].reset_index().to_csv('data/pval.csv', index=None, encoding='utf-8')
    globals()['correlation_matrix'].reset_index().to_csv('data/correlation_matrix.csv', index=None, encoding='utf-8')
    
    return globals()['correlation_matrix']

In [47]:
correlations(df=df)

/home/mcwaage1/qs/pycaret/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning:

An input array is constant; the correlation coefficient is not defined.



,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,concert_event_counts,concert_event_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_duration

In [214]:
bonferroni_rho.head(15)

,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,concert_event_counts,concert_event_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_duration

In [234]:
def make_correlation_heatmap(rho=rho):
    # https://community.plotly.com/t/correlation-heatmap-in-plotly/50884
    # https://community.plotly.com/t/correlation-plot-with-mask/29185/3
    
    title = 'Vitals vs Time 7 Day Rolling Average Correlation Heatmap'

    # # Mood Data
    # df = rho.loc[df_columns][df_columns]

    # # Mood Data vs Vitals
    # df = rho.loc[df_columns][vitals]

    # # Mood Data vs Time
    # df = rho.loc[df_counts][df_columns]
    # df = rho.loc[df_durations_seconds][df_columns]
    # df = rho.loc[df_durations_seconds_7_day_rolling_mean][df_columns]

    # # Time Data
    # df = rho.loc[df_counts][df_counts]
    # df = rho.loc[df_durations_seconds][df_durations_seconds]
    df = rho.loc[df_durations_seconds_7_day_rolling_mean][df_durations_seconds_7_day_rolling_mean]

    # # Vitals
    # df = rho.loc[vitals][vitals]

    # # Vitals vs Time
    # df = rho.loc[df_counts][vitals]
    # df = rho.loc[df_durations_seconds][vitals]
    df = rho.loc[df_durations_seconds_7_day_rolling_mean][vitals]
    
    # # Everything
    # df = rho

    # # only use mask for Mood vs Mood or Time vs Time etc.
    # mask = np.triu(np.ones_like(df, dtype=bool))
    # df = df.mask(mask)
    
    fig = go.Figure()
    fig = fig.add_trace(go.Heatmap(z=df.values,
                                   x=df.index,
                                   y=df.columns,
                                   #square=True,
                                   colorscale=px.colors.diverging.RdBu,
                                   reversescale=True,
                                   zmin=-1,
                                   zmax=1,
                                   xgap = 1,
                                   ygap = 1,
                                  ))
    
    app = jupyter_dash.JupyterDash(__name__,
                                   external_stylesheets=external_stylesheets,
                                   title=f"{title}")
    
    fig.update_xaxes(tickangle=-45,
                     #mirror='allticks', 
                     #side='top',
                    )
    fig.update_layout(title=title,
                      yaxis_autorange='reversed',
                      xaxis_showgrid=False,
                      yaxis_showgrid=False,
                      width=1200,
                      height=1000,
                     )
                  
    app.layout = html.Div([
        dcc.Graph(
            figure=fig,
            
            #https://plotly.com/python/setting-graph-size/
            #https://stackoverflow.com/questions/46287189/how-can-i-change-the-size-of-my-dash-graph
            style={'height': '98vh',
                   'width': '98vw',
                  },
            
    #         style={'height': '1100px',
    #                'width': '1600px',
    #               },
        ),
    ])

    title = title.replace('/', '_')
    fig.write_image(f"data/images/combined/correlation_heatmaps/{title.replace('/', '_')}.png")
    
    if __name__ == '__main__':
        app.run_server(mode='external', port=8100, debug=False)

In [235]:
make_correlation_heatmap(rho=bonferroni_rho)

Dash app running on http://127.0.0.1:8100/


# Bonferroni Correction

In [55]:
# https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html
from scipy.stats import pearsonr
import statsmodels.api #as sm
import statsmodels.formula.api #as smf

In [57]:
bonferroni_pvals = []
for col in correlation_matrix.columns:
    bonferroni_pvals.append(statsmodels.stats.multitest.multipletests(pval[col], 
                                                                      alpha=0.05, 
                                                                      method='bonferroni', 
                                                                      is_sorted=False, 
                                                                      returnsorted=False))


In [58]:
corrected_pvals = {}
for col in correlation_matrix.columns:
    for i in bonferroni_pvals:
        corrected_pvals[col] = bonferroni_pvals[0][1].tolist()
corrected_pvals = pd.DataFrame(corrected_pvals)
corrected_pvals['index'] = corrected_pvals.columns
corrected_pvals = corrected_pvals.set_index('index')
corrected_pvals

,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,concert_event_counts,concert_event_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_duration

In [59]:
bonferroni_rho = df.corr().round(3)
bonferroni_pval = corrected_pvals
bonferroni_p = bonferroni_pval.applymap(lambda x: ''.join(['*' for t in [0.001, 0.01, 0.05] if x<=t])) # can add more * for even smaller p value's e.g. 0.00001, 0.0001, in the list 
corrected_correlation_matrix = bonferroni_rho.astype(str) + bonferroni_p
with pd.option_context('display.max_rows', 50, 'display.max_columns', 50):
    display(corrected_correlation_matrix)

,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,...,drawing_durations_seconds_shifted,drawing_counts_shifted,drawing_durations_seconds_7_day_rolling_mean_shifted,traveling__durations_seconds_shifted,traveling__counts_shifted,traveling__durations_seconds_7_day_rolling_mean_shifted,drumming_durations_seconds_shifted,drumming_counts_shifted,drumming_durations_seconds_7_day_rolling_mean_shifted,data_analysis_durations_seconds_shifted,data_analysis_counts_shifted,data_analysis_durations_seconds_7_day_rolling_mean_shifted,concert_event_durations_seconds_shifted,concert_event_counts_shifted,concert_event_durations_seconds_7_day_rolling_mean_shifted,biking_durations_seconds_shifted,biking_counts_shifted,biking_durations_seconds_7_day_rolling_mean_shifted,dancing_durations_seconds_shifted,dancing_counts_shifted,dancing_durations_seconds_7_day_rolling_mean_shifted,Month,Day,Weekday,Week
Step Count,1.0***,-0.142***,0.436***,0.244***,0.331***,0.628***,-0.129***,0.014***,0.117***,0.337***,0.467***,0.162***,-0.134***,-0.059***,-0.06***,0.009***,-0.009***,-0.021***,0.019***,-0.028***,-0.028***,-0.025***,0.003***,-0.048***,0.018***,...,-0.009***,-0.021***,-0.036***,-0.008***,-0.007***,nan***,-0.005***,-0.016***,-0.044***,0.0***,0.003***,0.0***,0.038***,0.039***,-0.037***,0.032***,0.032***,nan***,0.034***,0.034***,nan***,0.048***,0.055***,0.169***,0.037***
Resting Heart Rate,-0.142***,1.0***,-0.063***,-0.004***,0.343***,-0.259***,0.475***,0.587***,0.399***,0.197***,-0.075***,0.044***,0.139***,0.066***,0.05***,-0.035***,-0.08***,-0.021***,0.048***,-0.071***,0.056***,-0.039***,-0.054***,0.02***,-0.023***,...,0.069***,0.075***,0.132***,-0.059***,-0.059***,nan***,-0.015***,-0.009***,-0.032***,-0.014***,-0.005***,-0.057***,-0.022***,-0.02***,0.045***,-0.049***,-0.049***,nan***,-0.021***,-0.021***,nan***,-0.042***,-0.037***,0.046***,-0.041***
Number of Floors,0.436***,-0.063***,1.0***,0.061***,0.174***,0.263***,-0.023***,0.037***,0.095***,0.181***,0.185***,0.03***,-0.068***,0.013***,-0.02***,0.011***,-0.003***,-0.004***,0.013***,0.014***,-0.042***,-0.018***,-0.005***,0.003***,-0.0***,...,0.0***,-0.004***,-0.011***,0.006***,0.007***,nan***,0.01***,0.004***,-0.03***,-0.009***,-0.007***,-0.017***,0.026***,0.018***,-0.013***,0.039***,0.039***,nan***,0.026***,0.026***,nan***,0.04***,0.005***,-0.007***,0.036***
Heart Rate Count,0.244***,-0.004***,0.061***,1.0***,-0.068***,0.015***,-0.086***,-0.089***,-0.081***,-0.048***,0.045***,0.01***,0.054***,0.004***,0.008***,0.016***,0.02***,0.01***,0.004***,0.021***,0.032***,0.022***,0.013***,0.028***,0.018***,...,0.01***,0.012***,0.018***,0.005***,0.005***,nan***,0.023***,0.024***,0.039***,0.006***,0.007***,0.017***,0.004***,0.004***,0.006***,0.007***,0.007***,nan***,0.003***,0.003***,nan***,-0.053***,0.042***,-0.038***,-0.045***
Heart Rate Mean,0.331***,0.343***,0.174***,-0.068***,1.0***,0.502***,0.415***,0.783***,0.922***,0.94***,0.394***,0.168***,-0.015***,-0.023***,0.044***,0.008***,-0.064***,-0.032***,0.013***,-0.058***,-0.03***,-0.057***,-0.008***,-0.018***,-0.043***,...,0.022***,0.013***,0.011***,-0.058***,-0.057***,nan***,-0.006***,-0.019***,-0.053***,-0.016***,-0.011***,-0.073***,0.021***,0.015***,0.016***,-0.011***,-0.011***,nan***,0.021***,0.021***,nan***,-0.047***,0.002***,-0.01***,-0.055***
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dancing_durations_seconds_7_day_rolling_mean_shifted,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,n

In [60]:
make_correlation_heatmap(rho=bonferroni_rho)

Dash app running on http://127.0.0.1:8100/


In [200]:
r_squared = bonferroni_rho*bonferroni_rho
r_squared

,Step Count,Resting Heart Rate,Number of Floors,Heart Rate Count,Heart Rate Mean,Heart Rate STD,Heart Rate Minimum,Heart Rate 25%,Heart Rate Median,Heart Rate 75%,Heart Rate Maximum,Mood,Pain,Tired,Sick,Anxiousness,Depressed,Suicidal,Self-Harm,Thoughts of Death,Repetitive Thoughts,Thoughts of Worthlessness,Thoughts of Hopelessness,Brain Sludge,Sluggish,Restless,Existential Crisis,Loss of Interest,Self Isolation,Paranoia,Elevated,Irritable/Angry,Loneliness,Libido,Weight,Weird,sleeping_durations_seconds,sleeping_counts,sleeping_durations_seconds_7_day_rolling_mean,internet_durations_seconds,internet_counts,internet_durations_seconds_7_day_rolling_mean,working_durations_seconds,working_counts,working_durations_seconds_7_day_rolling_mean,games_durations_seconds,games_counts,games_durations_seconds_7_day_rolling_mean,family_time_durations_seconds,family_time_counts,family_time_durations_seconds_7_day_rolling_mean,eating_durations_seconds,eating_counts,eating_durations_seconds_7_day_rolling_mean,programming_durations_seconds,programming_counts,programming_durations_seconds_7_day_rolling_mean,grooming_durations_seconds,grooming_counts,grooming_durations_seconds_7_day_rolling_mean,reading_durations_seconds,reading_counts,reading_durations_seconds_7_day_rolling_mean,gym_durations_seconds,gym_counts,gym_durations_seconds_7_day_rolling_mean,toilet_durations_seconds,toilet_counts,toilet_durations_seconds_7_day_rolling_mean,friends_time_durations_seconds,friends_time_counts,friends_time_durations_seconds_7_day_rolling_mean,studying_durations_seconds,studying_counts,studying_durations_seconds_7_day_rolling_mean,movies_tv_durations_seconds,movies_tv_counts,movies_tv_durations_seconds_7_day_rolling_mean,driving_durations_seconds,driving_counts,driving_durations_seconds_7_day_rolling_mean,walking_durations_seconds,walking_counts,walking_durations_seconds_7_day_rolling_mean,writing_durations_seconds,writing_counts,writing_durations_seconds_7_day_rolling_mean,cooking_durations_seconds,cooking_counts,cooking_durations_seconds_7_day_rolling_mean,class_durations_seconds,class_counts,class_durations_seconds_7_day_rolling_mean,cleaning_durations_seconds,cleaning_counts,cleaning_durations_seconds_7_day_rolling_mean,dishes_durations_seconds,dishes_counts,dishes_durations_seconds_7_day_rolling_mean,personal_durations_seconds,personal_counts,personal_durations_seconds_7_day_rolling_mean,dating_durations_seconds,dating_counts,dating_durations_seconds_7_day_rolling_mean,shopping_durations_seconds,shopping_counts,shopping_durations_seconds_7_day_rolling_mean,mycology_durations_seconds,mycology_counts,mycology_durations_seconds_7_day_rolling_mean,errands_and_appointments_durations_seconds,errands_and_appointments_counts,errands_and_appointments_durations_seconds_7_day_rolling_mean,club_social_group_durations_seconds,club_social_group_counts,club_social_group_durations_seconds_7_day_rolling_mean,drinking_durations_seconds,drinking_counts,drinking_durations_seconds_7_day_rolling_mean,gamedev_durations_seconds,gamedev_counts,gamedev_durations_seconds_7_day_rolling_mean,swimming_tanning_durations_seconds,swimming_tanning_counts,swimming_tanning_durations_seconds_7_day_rolling_mean,phone_durations_seconds,phone_counts,phone_durations_seconds_7_day_rolling_mean,keyboard_durations_seconds,keyboard_counts,keyboard_durations_seconds_7_day_rolling_mean,journaling_durations_seconds,journaling_counts,journaling_durations_seconds_7_day_rolling_mean,drawing_durations_seconds,drawing_counts,drawing_durations_seconds_7_day_rolling_mean,traveling__durations_seconds,traveling__counts,traveling__durations_seconds_7_day_rolling_mean,drumming_durations_seconds,drumming_counts,drumming_durations_seconds_7_day_rolling_mean,data_analysis_durations_seconds,data_analysis_counts,data_analysis_durations_seconds_7_day_rolling_mean,concert_event_durations_seconds,concert_event_counts,concert_event_durations_seconds_7_day_rolling_mean,biking_durations_seconds,biking_counts,biking_duration